# Cleaning

In [1]:
import pandas as pd
import re
from library.sb_utils import save_file
from nltk.corpus import stopwords
pd.options.mode.chained_assignment = None  # default='warn'

Alright, let's see what we're are working with...

In [2]:
data = pd.read_csv("../data/fake_job_postings.csv")
data.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


A lot of info here. I can already see a lot of NaNs. Lets check out the datatypes.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

Looks like `job_id` can be set as the numeric index. `salary_range` should also be inspected since it's data type is an object, and I was expecting a numeric.

I'll start with `job_id`. `job_id` *should* be unique, so lets check on that first and make it the index if it is indeed unique.

In [4]:
data['job_id'].is_unique

True

In [5]:
clean_data = data.set_index('job_id')

In [6]:
clean_data.head(2)

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
job_id,,,,,,,,,,,,,,,,,
1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0


Looking good so far. Time to inspect `salary_range`.

In [7]:
salary_range = clean_data['salary_range']
salary_range

job_id
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
        ... 
17876    NaN
17877    NaN
17878    NaN
17879    NaN
17880    NaN
Name: salary_range, Length: 17880, dtype: object

Lots of NaN here... what happens if we drop those...?

In [8]:
salary_range.dropna()

job_id
7          20000-28000
11       100000-120000
16       120000-150000
24       100000-120000
32         50000-65000
             ...      
17845              0-0
17850     80000-100000
17866      18000-20000
17868      18000-19000
17875     80000-100000
Name: salary_range, Length: 2868, dtype: object

Ah, okay, I see now. I'll create three new columns based on `salary_range`: `min_salary`, `max_salary` and a redefinition of `salary_range` (which will be the difference of the two).

In [9]:
clean_data['min_salary'] = 0
clean_data['max_salary'] = 0

In [10]:
clean_data['salary_range'] = clean_data['salary_range'].str.split('-')

In [11]:
for i in range(len(clean_data)):
    try:
        value = clean_data['salary_range'].iloc[i]
        if type(value) is list:
            clean_data['min_salary'].iloc[i] = int(value[0])
            clean_data['max_salary'].iloc[i] = int(value[1])
            clean_data['salary_range'].iloc[i] = int(value[1]) - int(value[0])
        else:
            clean_data['salary_range'].iloc[i] = 0
            
    except ValueError:
        clean_data['salary_range'].iloc[i] = 0
    except IndexError:
        clean_data['salary_range'].iloc[i] = 0
clean_data['salary_range'] = clean_data['salary_range'].astype(str).astype(int)

In [12]:
clean_data['salary_range'].dtype

dtype('int64')

In [13]:
clean_data[clean_data['salary_range']!=0].head(3)

,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,min_salary,max_salary
job_id,,,,,,,,,,,,,,,,,,,
7,Head of Content (m/f),"DE, BE, Berlin",ANDROIDPIT,8000,"Founded in 2009, the Fonpit AG rose with its i...",Your Responsibilities: Manage the English-spea...,Your Know-How: ...,Your Benefits: Being part of a fast-growing co...,0,1,1,Full-time,Mid-Senior level,Master's Degree,Online Media,Management,0,20000,28000
11,ASP.net Developer Job opportunity at United St...,"US, NJ, Jersey City",NaN,20000,NaN,Position : #URL_86fd830a95a64e2b30ceed829e63fd...,Position : #URL_86fd830a95a64e2b30ceed829e63fd...,Benefits - FullBonus Eligible - YesInterview T...,0,0,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,0,100000,120000
16,VP of Sales - Vault Dragon,"SG, 01, Singapore",Sales,30000,Jungle Ventures is the leading Singapore based...,About Vault Dragon Vault Dragon is Dropbox for...,Key Superpowers3-5 years of high-pressure sale...,"Basic: SGD 120,000Equity negotiable for a rock...",0,1,1,Full-time,Executive,Bachelor's Degree,Facilities Services,Sales,0,120000,150000


Now, I'll drop the `min_salary` and `max_salary` features I created earlier.

In [14]:
clean_data.drop(['min_salary', 'max_salary'], axis=1, inplace=True)

`salary_range` and `job_id` have now been taken care of.

### Cleaning null values

I can still see a bunch of NaNs. Let's take a look at the those now.

In [15]:
clean_data["required_experience"].value_counts(dropna=False)

NaN                 7050
Mid-Senior level    3809
Entry level         2697
Associate           2297
Not Applicable      1116
Director             389
Internship           381
Executive            141
Name: required_experience, dtype: int64

Plenty of NaN values in `required_experience`. It wouldn't be smart to drop them all since there are so many. I'll fill them with an actual string value for now.

In [16]:
clean_data["required_experience"] = clean_data["required_experience"].fillna(value='Not Applicable')
clean_data["required_experience"].value_counts(dropna=False)

Not Applicable      8166
Mid-Senior level    3809
Entry level         2697
Associate           2297
Director             389
Internship           381
Executive            141
Name: required_experience, dtype: int64

Much better. Now for `required_education`.

In [17]:
clean_data["required_education"].value_counts(dropna=False)

NaN                                  8105
Bachelor's Degree                    5145
High School or equivalent            2080
Unspecified                          1397
Master's Degree                       416
Associate Degree                      274
Certification                         170
Some College Coursework Completed     102
Professional                           74
Vocational                             49
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree                     6
Name: required_education, dtype: int64

Same issue as before. I'll set the NaNs to Unspecified.

In [18]:
clean_data["required_education"] = clean_data["required_education"].fillna(value='Unspecified')
clean_data["required_education"].value_counts(dropna=False)

Unspecified                          9502
Bachelor's Degree                    5145
High School or equivalent            2080
Master's Degree                       416
Associate Degree                      274
Certification                         170
Some College Coursework Completed     102
Professional                           74
Vocational                             49
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree                     6
Name: required_education, dtype: int64

Now for `employment_type`.

In [19]:
clean_data["employment_type"].value_counts(dropna=False)

Full-time    11620
NaN           3471
Contract      1524
Part-time      797
Temporary      241
Other          227
Name: employment_type, dtype: int64

In [20]:
clean_data["employment_type"] = clean_data["employment_type"].fillna(value='Other')
clean_data["employment_type"].value_counts(dropna=False)

Full-time    11620
Other         3698
Contract      1524
Part-time      797
Temporary      241
Name: employment_type, dtype: int64

Next up, `department`. 

In [21]:
data["department"].value_counts(dropna=False)

NaN                                  11547
Sales                                  551
Engineering                            487
Marketing                              401
Operations                             270
                                     ...  
IT Administrator / Developer             1
130 - Nutritional Yeast Packaging        1
Business: Support                        1
Visionary Engineering                    1
Client Relations                         1
Name: department, Length: 1338, dtype: int64

Hmmm... there are a lot of null values here. Not enough data in the column. I'll drop it later.

Almost there...

In [22]:
clean_data["industry"].value_counts(dropna=False)

NaN                                    4903
Information Technology and Services    1734
Computer Software                      1376
Internet                               1062
Marketing and Advertising               828
                                       ... 
Ranching                                  1
Museums and Institutions                  1
Shipbuilding                              1
Alternative Dispute Resolution            1
Sporting Goods                            1
Name: industry, Length: 132, dtype: int64

Hmmm... a lot of nulls again. I'll take note and drop this column later.

Last up, `function`.

In [23]:
clean_data["function"].value_counts(dropna=False)

NaN                       6455
Information Technology    1749
Sales                     1468
Engineering               1348
Customer Service          1229
Marketing                  830
Administrative             630
Design                     340
Health Care Provider       338
Other                      325
Education                  325
Management                 317
Business Development       228
Accounting/Auditing        212
Human Resources            205
Project Management         183
Finance                    172
Consulting                 144
Writing/Editing            132
Art/Creative               132
Production                 116
Product Management         114
Quality Assurance          111
Advertising                 90
Business Analyst            84
Data Analyst                82
Public Relations            76
Manufacturing               74
General Business            68
Research                    50
Legal                       47
Strategy/Planning           46
Training

A lot of null values here too, but I can group them with Other.

In [24]:
clean_data["function"] = clean_data["function"].fillna(value='Other')
clean_data["function"].value_counts(dropna=False)

Other                     6780
Information Technology    1749
Sales                     1468
Engineering               1348
Customer Service          1229
Marketing                  830
Administrative             630
Design                     340
Health Care Provider       338
Education                  325
Management                 317
Business Development       228
Accounting/Auditing        212
Human Resources            205
Project Management         183
Finance                    172
Consulting                 144
Art/Creative               132
Writing/Editing            132
Production                 116
Product Management         114
Quality Assurance          111
Advertising                 90
Business Analyst            84
Data Analyst                82
Public Relations            76
Manufacturing               74
General Business            68
Research                    50
Legal                       47
Strategy/Planning           46
Training                    38
Supply C

Finally done with looking for null values. Time to drop `department` and `industry`.

In [25]:
clean_data=clean_data.drop(['department', 'industry'], axis=1)

Let's see how the data looks now. 

In [26]:
clean_data.head(3)

,title,location,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,function,fraudulent
job_id,,,,,,,,,,,,,,,
1,Marketing Intern,"US, NY, New York",0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,Unspecified,Marketing,0
2,Customer Service - Cloud Video Production,"NZ, , Auckland",0,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,Unspecified,Customer Service,0
3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,Other,Not Applicable,Unspecified,Other,0


Now its time to fill in the null values for the columns that contain substantial textual data.

Let's see how many nulls exsist in `title`, `location`, `company_profile`, `description`, `requirements` and `benefits`.

In [27]:
title_null = sum(clean_data['title'].isnull())
location_null = sum(clean_data['location'].isnull())
company_profile_null = sum(clean_data['company_profile'].isnull())
description_null = sum(clean_data['description'].isnull())
requirements_null = sum(clean_data['requirements'].isnull())
benefits_null = sum(clean_data['benefits'].isnull())
print(title_null, location_null, company_profile_null, description_null, requirements_null, benefits_null)

0 346 3308 1 2695 7210


Plenty to fill in here. Let's just fill these in with empty strings.

In [28]:
clean_data['location'] = clean_data['location'].fillna("")
clean_data['company_profile'] = clean_data['company_profile'].fillna("")
clean_data['description'] = clean_data['description'].fillna("")
clean_data['requirements'] = clean_data['requirements'].fillna("")
clean_data['benefits'] = clean_data['benefits'].fillna("")

In [29]:
title_null = sum(clean_data['title'].isnull())
location_null = sum(clean_data['location'].isnull())
company_profile_null = sum(clean_data['company_profile'].isnull())
description_null = sum(clean_data['description'].isnull())
requirements_null = sum(clean_data['requirements'].isnull())
benefits_null = sum(clean_data['benefits'].isnull())
print(title_null, location_null, company_profile_null, description_null, requirements_null, benefits_null)

0 0 0 0 0 0


#### Combining all the text columns into one

Now its time to do something with these text columns. For sake of processing and simplicity, I am going to combine all the text columns into one `text` column.

In [30]:
clean_data['text'] = clean_data[['title', 'location', 'company_profile', 'description', 
                                 'requirements', 'benefits']].apply('-'.join, axis=1)
clean_data = clean_data.drop(['title', 'location', 'company_profile', 'description', 
                              'requirements', 'benefits'], axis=1)

In [31]:
clean_data.head(2)

,salary_range,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,function,fraudulent,text
job_id,,,,,,,,,,
1,0,0,1,0,Other,Internship,Unspecified,Marketing,0,"Marketing Intern-US, NY, New York-We're Food52..."
2,0,0,1,0,Full-time,Not Applicable,Unspecified,Customer Service,0,"Customer Service - Cloud Video Production-NZ, ..."


Now that there is a single text column, I will remove all the stopwords and special characters.

In [32]:
clean_data['text'] = clean_data['text'].str.lower()
stop = stopwords.words('english')
clean_data['text'] = clean_data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
clean_data['text'] = clean_data['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
clean_data['text'] = clean_data['text'].map(lambda x: re.sub(r'\W+', ' ', x))

In [33]:
clean_data.head(3)

,salary_range,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,function,fraudulent,text
job_id,,,,,,,,,,
1,0,0,1,0,Other,Internship,Unspecified,Marketing,0,marketing intern us ny new york we re food52 w...
2,0,0,1,0,Full-time,Not Applicable,Unspecified,Customer Service,0,customer service cloud video production nz auc...
3,0,0,1,0,Other,Not Applicable,Unspecified,Other,0,commissioning machinery assistant cma us ia we...


This looks good so far. The next step will be EDA. Hopefully, more will be learned there and I can start picking the data apart some more.

##### Save File

In [34]:
save_file(clean_data, 'clean_data.csv', '../data')

A file already exists with this name.

Do you want to overwrite? (Y/N)Y
Writing file.  "../data/clean_data.csv"
